In [1]:
%%writefile mapper-tf.py
from __future__ import print_function
import sys
import re


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

path = 'stop_words_en.txt'

stopWords = set()

with open(path) as stopWordsFile:
    for line in stopWordsFile:
        try:         
            stopWords.add(unicode(line.strip()))
        except ValueError as e:
            continue

import string

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    words = re.split("\W*\s+\W*", text.lower(), flags=re.UNICODE)
    
    wordsInArticle = 0
    
    for word in words:
        word = word.strip(string.punctuation)
        if word in stopWords:
            print("reporter:counter:Wiki stats,Stop words found,%d" % 1, file=sys.stderr)
            continue
        if word == "":
            print("reporter:counter:Wiki stats,Null words found,%d" % 1, file=sys.stderr)
            continue
        wordsInArticle += 1
        print("reporter:counter:Wiki stats,Total words found,%d" % 1, file=sys.stderr)
        print(article_id, word.strip(string.punctuation), 1, sep='\t')
    print(article_id, "!wordsInArticle", wordsInArticle, sep='\t')



Overwriting mapper-tf.py


In [27]:
%%writefile reducer-tf.py
import sys


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

current_article_id = None
current_word = None
word_sum, wordsInArticle = 0, 0

for line in sys.stdin:
    try:
        article_id, word, count = unicode(line.strip()).split('\t')
        #print "after parse:", article_id, word, count
        if article_id == "" or word == "" or count == "":   # sanity check
            continue
        count = int(count)
    except ValueError as e:
        #print "Parse error:", e
        continue

    if current_article_id != article_id:
        if current_article_id != None and current_word != '!wordsInArticle':
            print "%s\t%s\t%f" % (current_article_id, current_word, 1.0*word_sum/wordsInArticle)
            word_sum = 0
            current_word = word
        wordsInArticle = count  # first article string is always words in article count
        current_article_id = article_id
        #print "wordsInArticle:", word, wordsInArticle
        continue
        
    if current_word != word :
        if current_word != None and current_word != '!wordsInArticle': 
            print "%s\t%s\t%f" % (current_article_id, current_word, 1.0*word_sum/wordsInArticle)
            #print "%s\t%s\t%d\t%d" % (current_article_id, current_word, word_sum, wordsInArticle)
        word_sum = 0
        current_word = word
    word_sum += count

if current_article_id and word != '!wordsInArticle':
    print "%s\t%s\t%f" % (current_article_id, word, 1.0*word_sum/wordsInArticle)


Overwriting reducer-tf.py


In [3]:
%%writefile mapper-idf.py
from __future__ import print_function
import sys

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

for line in sys.stdin:
    _, word, _ = unicode(line.strip()).split('\t')
    print("reporter:counter:Idf stats,Total words found,%d" % 1, file=sys.stderr)
    print(word, 1, sep='\t')


Overwriting mapper-idf.py


In [4]:
%%writefile reducer-idf.py
import sys
from math import log

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

current_key = None
word_sum = 0
for line in sys.stdin:
    try:
        key, count = unicode(line.strip()).split('\t', 1)
        if key == "" or count == "":    # sanity check
            continue
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%f" % (current_key, (1/log(1 + word_sum)))
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%f" % (current_key, (1/log(1 + word_sum)))

Overwriting reducer-idf.py


In [36]:
%%writefile mapper-tf-idf.py
from __future__ import print_function
import sys


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode


for line in sys.stdin:
    inVal = unicode(line.strip()).split('\t')
    inValLen = len(inVal)
    #print(inValLen, inVal)
    try:
        if inValLen == 2: # idf data
            print("reporter:counter:Wiki stats,idf words,%d" % 1, file=sys.stderr)
            print(inVal[0], "idf", float(inVal[1]), sep='\t')
        elif inValLen == 3: # tf data
            print("reporter:counter:Wiki stats,tf words,%d" % 1, file=sys.stderr)
            print(inVal[1], "tf", inVal[0], float(inVal[2]), sep='\t')
        else:
            print("reporter:counter:Wiki stats,tf idf parsing errors,%d" % 1, file=sys.stderr)
    except ValueError as e:
        print("reporter:counter:Wiki stats,tf idf to float errors,%d" % 1, file=sys.stderr)
        continue


Overwriting mapper-tf-idf.py


In [42]:
%%writefile reducer-tf-idf.py
import sys
from math import log


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

for line in sys.stdin:
    inVal = unicode(line.strip()).split('\t')
    if (len(inVal) < 2 and len(inVal) > 4):      # sanity check
        continue
    try:
        if inVal[1] == 'idf':
            idfWord, _, idfDt = inVal
            idfDt = float(idfDt)
            continue
        elif inVal[1] == 'tf':
            tfWord, _, article_id, tfNt = inVal
            tfWord = tfWord
            tfNt = float(tfNt)
        else:
            continue
    except ValueError as e:
        continue
    if idfWord == tfWord:
        print "%s\t%s\t%f" % (idfWord, article_id, tfNt*idfDt)


Overwriting reducer-tf-idf.py


In [28]:
%%bash

OUT_DIR_TF="tf_result"
#OUT_DIR="/data/wiki/test/tf-out"
NUM_REDUCERS=4

hdfs dfs -rm -r -skipTrash ${OUT_DIR_TF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.job.output.key.partitioner.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedPartitioner \
    -D mapreduce.job.name="Tf-Idf tf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-tf.py,reducer-tf.py,/datasets/stop_words_en.txt  \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D mapreduce.partition.keycomparator.options="-k1,2" \
    -mapper "python mapper-tf.py" \
    -reducer "python reducer-tf.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR_TF} > /dev/null


19/02/06 10:09:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 10:09:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 10:09:21 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/06 10:09:21 INFO mapreduce.JobSubmitter: number of splits:2
19/02/06 10:09:21 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549443035368_0011
19/02/06 10:09:21 INFO impl.YarnClientImpl: Submitted application application_1549443035368_0011
19/02/06 10:09:22 INFO mapreduce.Job: The url to track the job: http://427e53c98640:8088/proxy/application_1549443035368_0011/
19/02/06 10:09:22 INFO mapreduce.Job: Running job: job_1549443035368_0011
19/02/06 10:09:28 INFO mapreduce.Job: Job job_1549443035368_0011 running in uber mode : false
19/02/06 10:09:28 INFO mapreduce.Job:  map 0% reduce 0%
19/02/06 10:09:44 INFO mapreduce.Job:  map 20% reduce 0%
19/02/06 10:09:50 INFO mapreduce.Job:  map 30% reduce 0%
19/02/06 10:09:56 INFO 

In [29]:
! hdfs dfs -getmerge tf_result/* tf.csv

In [30]:
!grep wordsInArticle tf.csv

look at articles

8234	!wordsInArticle	0.000000
5528	!wordsInArticle	0.000000

In [31]:
%%bash
#OUT_DIR_IDF="idf_result_"$(date +"%s%6N")
OUT_DIR_TF="tf_result"
OUT_DIR_IDF="idf_result"
NUM_REDUCERS=3

hdfs dfs -rm -r -skipTrash ${OUT_DIR_IDF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Tf-Idf idf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-idf.py,reducer-idf.py \
    -mapper "python mapper-idf.py" \
    -reducer "python reducer-idf.py" \
    -input ${OUT_DIR_TF} \
    -output ${OUT_DIR_IDF} > /dev/null



rm: `idf_result': No such file or directory
19/02/06 10:15:21 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 10:15:21 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 10:15:22 INFO mapred.FileInputFormat: Total input files to process : 4
19/02/06 10:15:22 INFO mapreduce.JobSubmitter: number of splits:4
19/02/06 10:15:22 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549443035368_0012
19/02/06 10:15:23 INFO impl.YarnClientImpl: Submitted application application_1549443035368_0012
19/02/06 10:15:23 INFO mapreduce.Job: The url to track the job: http://427e53c98640:8088/proxy/application_1549443035368_0012/
19/02/06 10:15:23 INFO mapreduce.Job: Running job: job_1549443035368_0012
19/02/06 10:15:29 INFO mapreduce.Job: Job job_1549443035368_0012 running in uber mode : false
19/02/06 10:15:29 INFO mapreduce.Job:  map 0% reduce 0%
19/02/06 10:15:48 INFO mapreduce.Job:  map 62% reduce 0%
19/02/06 10:15:49 INFO mapreduce.Job

In [32]:
! hdfs dfs -ls idf_result

Found 4 items
-rw-r--r--   1 jovyan supergroup          0 2019-02-06 10:16 idf_result/_SUCCESS
-rw-r--r--   1 jovyan supergroup    2758601 2019-02-06 10:15 idf_result/part-00000
-rw-r--r--   1 jovyan supergroup    2769065 2019-02-06 10:15 idf_result/part-00001
-rw-r--r--   1 jovyan supergroup    2758034 2019-02-06 10:16 idf_result/part-00002


In [33]:
! hdfs dfs -getmerge idf_result/* idf.csv

In [34]:
! head idf.csv

0%however	1.442695
0&\mathrm{if	1.442695
0&\mbox{if	1.442695
0)).(1	1.442695
0).in	1.442695
0).one	1.442695
0+1	1.442695
0,0	0.455120
0,00	1.442695
0,03	1.442695


In [35]:
! tail idf.csv

𐑈	1.442695
𐑋	1.442695
𐑎	1.442695
𐡁	1.442695
𐡄	1.442695
𐡇	1.442695
𐡊	1.442695
𐡍	1.442695
𐡐	1.442695
𐡓	1.442695


In [43]:
%%bash
OUT_DIR_TF="tf_result"
OUT_DIR_IDF="idf_result"
#OUT_DIR_TF_IDF="tf_idf_result_"$(date +"%s%6N")
OUT_DIR_TF_IDF="tf-idf-out"
NUM_REDUCERS=3

hdfs dfs -rm -r -skipTrash ${OUT_DIR_TF_IDF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.job.output.key.partitioner.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedPartitioner \
    -D mapreduce.job.name="Tf-Idf tf*idf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-tf-idf.py,reducer-tf-idf.py \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D stream.num.map.output.key.fields=2 \
    -mapper "python mapper-tf-idf.py" \
    -reducer "python reducer-tf-idf.py" \
    -input ${OUT_DIR_TF},${OUT_DIR_IDF} \
    -output ${OUT_DIR_TF_IDF} > /dev/null



rm: `tf-idf-out': No such file or directory
19/02/06 11:12:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 11:12:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 11:12:21 INFO mapred.FileInputFormat: Total input files to process : 7
19/02/06 11:12:21 INFO mapreduce.JobSubmitter: number of splits:7
19/02/06 11:12:21 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549443035368_0015
19/02/06 11:12:22 INFO impl.YarnClientImpl: Submitted application application_1549443035368_0015
19/02/06 11:12:22 INFO mapreduce.Job: The url to track the job: http://427e53c98640:8088/proxy/application_1549443035368_0015/
19/02/06 11:12:22 INFO mapreduce.Job: Running job: job_1549443035368_0015
19/02/06 11:12:28 INFO mapreduce.Job: Job job_1549443035368_0015 running in uber mode : false
19/02/06 11:12:28 INFO mapreduce.Job:  map 0% reduce 0%
19/02/06 11:12:44 INFO mapreduce.Job:  map 14% reduce 0%
19/02/06 11:12:48 INFO mapreduce.Job

In [13]:
%%bash

OUT_DIR_TF="tf_result_"$(date +"%s%6N")
#OUT_DIR="/data/wiki/test/tf-out"
NUM_REDUCERS=3

hdfs dfs -rm -r -skipTrash ${OUT_DIR_TF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.job.output.key.partitioner.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedPartitioner \
    -D mapreduce.job.name="Tf-Idf tf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-tf.py,reducer-tf.py,/datasets/stop_words_en.txt  \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D mapreduce.partition.keycomparator.options="-k1,2" \
    -mapper "python mapper-tf.py" \
    -reducer "python reducer-tf.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR_TF} > /dev/null


OUT_DIR_IDF="idf_result_"$(date +"%s%6N")
#OUT_DIR="/data/wiki/test/idf-out"
NUM_REDUCERS=3

hdfs dfs -rm -r -skipTrash ${OUT_DIR_IDF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Tf-Idf idf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-idf.py,reducer-idf.py \
    -mapper "python mapper-idf.py" \
    -reducer "python reducer-idf.py" \
    -input ${OUT_DIR_TF} \
    -output ${OUT_DIR_IDF} > /dev/null


OUT_DIR_TF_IDF="tf_idf_result_"$(date +"%s%6N")
#OUT_DIR="/data/wiki/test/tf-idf-out"
NUM_REDUCERS=3

hdfs dfs -rm -r -skipTrash ${OUT_DIR_TF_IDF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.job.output.key.partitioner.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedPartitioner \
    -D mapreduce.job.name="Tf-Idf tf*idf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-tf-idf.py,reducer-tf-idf.py \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D stream.num.map.output.key.fields=2 \
    -mapper "python mapper-tf-idf.py" \
    -reducer "python reducer-tf-idf.py" \
    -input ${OUT_DIR_TF},${OUT_DIR_IDF} \
    -output ${OUT_DIR_TF_IDF} > /dev/null


#hdfs dfs -cat ${OUT_DIR_TF_IDF} | grep labor

rm: `tf_result_1549446076952079': No such file or directory
19/02/06 09:41:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 09:41:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 09:41:21 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/06 09:41:21 INFO mapreduce.JobSubmitter: number of splits:2
19/02/06 09:41:21 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549443035368_0006
19/02/06 09:41:22 INFO impl.YarnClientImpl: Submitted application application_1549443035368_0006
19/02/06 09:41:22 INFO mapreduce.Job: The url to track the job: http://427e53c98640:8088/proxy/application_1549443035368_0006/
19/02/06 09:41:22 INFO mapreduce.Job: Running job: job_1549443035368_0006
19/02/06 09:41:28 INFO mapreduce.Job: Job job_1549443035368_0006 running in uber mode : false
19/02/06 09:41:28 INFO mapreduce.Job:  map 0% reduce 0%
19/02/06 09:41:45 INFO mapreduce.Job:  map 19% reduce 0%
19/02/06 09:41:51 IN